# Evaluate file

In [2]:
from class_utils import *
import pickle


In [17]:
iteration =  "in-favor"
filepath = "/Users/amyburkhardt/Documents/NLP/Code/combining-machine-qual/"
ev_filename = "{}/new_data/2015-April.txt".format(filepath)

# maps estimator model to pickled scikit-learn model file name:
eval_dict = {
    "SVC-{}".format(iteration): "{}/saved_models/best_svc_{}.pickle".format(filepath,iteration)
}

In [18]:
# load estimator models from disk
models = {}
for model, file in eval_dict.items():
    with open(file, 'rb') as f:
        loaded_model = pickle.load(f)
    models[model] = loaded_model

In [19]:
models

{'SVC-in-favor': Pipeline(memory=None,
      steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=0.45555555555555555, max_features=None,
         min_df=1, ngram_range=(1, 3),
         preprocessor=<function no...,
   max_iter=-1, probability=False, random_state=None, shrinking=True,
   tol=0.001, verbose=False))])}

In [20]:
# get results for each estimatator model
results = {}
for model, estimator in models.items():
    results[model] = evaluate_file(ev_filename, models[model])

In [21]:
# print result counts

for model, result in results.items():
    nneg = len(result[0])
    npos = len(result[1])
    print("Counts for %s:" % model)
    print("Number of NEG docs: %d" % nneg)
    print("Number of POS docs: %d" % npos)
    print("Percent POS: %.2f" % (npos/(npos+nneg)*100))
    print()

Counts for SVC-in-favor:
Number of NEG docs: 12631
Number of POS docs: 470
Percent POS: 3.59



In [22]:
# first three results of each class
for model, result in results.items():
    print("Top 3 of each class for %s:" % model)
    for c in result.keys():
        print("Class %s: " % str(c))
        for tweet in result[c][:3]:
            print(tweet[1])
        print()

Top 3 of each class for SVC-in-favor:
Class 0: 
Jeb Bush should 'rethink' running for President. Jeb Bush: Parents should 'rethink' opting out of Common Core tests http://www.washingtonexaminer.com/jeb-bush-parents-should-rethink-opting-out-of-common-core-tests/article/2563854#.VUMUB9QfOTo.twitter …

Legal Implications of High-Stakes Assessment: What States [ #SmarterBalanced & #PARCC] Should Know // v ERIC @scoopit http://sco.lt/8YyD5N

Fairtest Reacts to Ongoing Computer-Administered #Testing Problems: Exams Crash in 9 States http://sco.lt/9My8cD  // #SBAC #education


Class 1: 
According to a poll by the Economist only 30% of parents think #optout should be an option http://goo.gl/DWpA05 @BorowitzReport

Opting Out of Testing http://www.eagleforum.org/publications/educate/apr15/opting-out-of-testing.html#.VULDJ1cWbp0.twitter … via @EagleForum

Debates over testing crossing the line as opt outs put NM at risk http://goo.gl/SeYdGr @teacherromeronm




In [23]:
# save results to disk
for model, result in results.items():
    for c in result.keys():
        out_filename = "%sclassified_new_data/%s_%s_out_class%d.txt" % (filepath,iteration, model, c)
        with open(out_filename, 'w') as f:
            for line in result[c]:
                f.write("%s\t%s" % line)